# **Data Streaming using PySpark [CN7030]**

**`Dr Amin Karami, UEL Docklands Campus, March 2022`**

`E: a.karami@uel.ac.uk`

`W: www.aminkarami.com`

---

**foreachRDD(func)**:	The most generic output operator that applies a function, `func`, to each RDD generated from the stream. This function should push the data in each RDD to an external system, such as saving the RDD to files, or writing it over the network to a database. Note that the function func is executed in the driver process running the streaming application, and will usually have RDD actions in it that will force the computation of the streaming RDDs.


In [2]:
# Load Spark engine
import findspark
findspark.init()

In [3]:
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

In [4]:
sc = SparkContext(master="local[*]", appName="Output_Stream_example")
sc.setLogLevel("ERROR")

ssc = StreamingContext(sc, 5)
ssc.checkpoint("checkpoint")

22/03/28 14:14:24 WARN Utils: Your hostname, Predator-G3572 resolves to a loopback address: 127.0.1.1; using 172.29.43.74 instead (on interface eth0)
22/03/28 14:14:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/28 14:14:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/28 14:14:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
lines = ssc.socketTextStream("localhost", 7000)
# open cmd and type: nc -lk 7000

In [6]:
def rdd_print(rdd):
    if not rdd.isEmpty():
        calc = rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: int(a)+int(b))
        print(calc.collect())

In [7]:
words = lines.flatMap(lambda line: line.split(" "))
words.foreachRDD(rdd_print)

In [8]:
ssc.start()

[('helo', 1)]


[('i', 1), ('am', 1), ('noman', 1)]


[('helo', 1), ('gain', 1)]


[('helo', 1)]


[('gg', 1)]


[('gg', 1), ('ggg', 2)]


[('gg', 3)]


[('gg', 1)]


[('gg', 2)]


In [9]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

22/03/28 14:17:28 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3